# Initialization

In [1]:
### 0.1 Use development scripts or the package installed from pip
use_development = True
if use_development:
    import sys
    import os
    # Adjust the sys.path to include the project root directory
    project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    from src.pybalmorel import MainResults
    from src.pybalmorel.utils import symbol_to_df
else:
    from pybalmorel import MainResults
    from pybalmorel.utils import symbol_to_df

import ctypes

# Explicitly load the GAMS DLL
try:
    ctypes.windll.LoadLibrary("C:/GAMS/48/gmdcclib64.dll")
    print("GAMS DLL loaded successfully")
except Exception as e:
    print(f"Failed to load GAMS DLL: {e}")

GAMS DLL loaded successfully


## Loading in the GDX file

In [2]:
res1 = MainResults(files='Optiflow_MainResults1.gdx', paths='files')

c:\Users\anton\OneDrive - Danmarks Tekniske Universitet\Master_Projekt\PyBal\pybalmorel-master\PostProcessing\files\Optiflow_MainResults1.gdx


### Creating DataFrames

In [5]:
import pandas as pd
# Getting a result in DataFrame format
df_res1 = pd.DataFrame(res1.get_result('VFLOW_Opti_C'))
df_res1.head()

,Scenario,Y,CCC,IPROCFROM,IPROCTO,FLOW,Value
0,Optiflow_1,2050,DENMARK,HYDROGEN,Hydrogen_GRID_IN,HYDROGEN_FLOW,64.198239
1,Optiflow_1,2050,DENMARK,EMethanolBuffer,EMethanol_Eff,EMETHANOLFLOW,21.951930
2,Optiflow_1,2050,DENMARK,EMethanolBuffer,Money_buffer_T,OPERATIONCOST,0.219519
3,Optiflow_1,2050,DENMARK,AmmoniaBuffer,Ammonia_Eff,AMMONIA_FLOW,38.515847
4,Optiflow_1,2050,DENMARK,AmmoniaBuffer,Money_buffer_T,OPERATIONCOST,0.385158


#### Filtering data example

In [6]:
#Filter the data frame to be only the ones with "CO2" contained in the "IPROCFROM" column and denmark from the CCC column
df_res1Filtered = df_res1[df_res1['IPROCFROM'].str.contains('CO2') & df_res1['CCC'].str.contains('DENMARK')].copy()
df_res1Filtered

# Calculate Sequestered CO2
# Rows where IPROCTO indicates sequestration (e.g., CO2_Seq_50 or CO2_Seq_Sum)
sequestered = df_res1Filtered[df_res1Filtered['IPROCTO'].str.contains('CO2_Seq')]['Value'].sum()

# Calculate Utilized CO2
# Rows where IPROCTO indicates utilization (e.g., CO2_Use or Ethanol_synthesis)
utilized = df_res1Filtered[df_res1Filtered['IPROCTO'].str.contains('CO2_Use|Ethanol|Methanol')]['Value'].sum()

# Calculate Emitted CO2
# Rows where IPROCTO indicates emissions (e.g., CO2_Grid_Out or other outputs)
emitted = df_res1Filtered[df_res1Filtered['IPROCTO'].str.contains('CO2_Grid_Out|CO2_FLOW_OUT')]['Value'].sum()

# Print results
print(f"Sequestered CO₂: {sequestered:.3f} MtCO₂")
print(f"Utilized CO₂: {utilized:.3f} MtCO₂")

Sequestered CO₂: 1.597 MtCO₂
Utilized CO₂: 3.028 MtCO₂
